In [50]:
import os
import cv2 as cv
import argparse
from sklearn.svm import LinearSVC
from skimage import feature
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from utils import read_img
from sklearn.metrics import f1_score
import joblib
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler
import helper
from sklearn.metrics import f1_score

In [2]:
train_image_paths = []
train_labels = []
# get all the image folder paths
Categories=['black widow', 'captain america', 'doctor strange', 'hulk', 'ironman', 'loki', 'spider-man', 'thanos']
datadir='dataset\\train' 

for i in Categories:
    
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        train_image_paths.append(os.path.join(path,img))
        train_labels.append(Categories.index(i))
    print(f'loaded category:{i} successfully')

loading... category : black widow
loaded category:black widow successfully
loading... category : captain america
loaded category:captain america successfully
loading... category : doctor strange
loaded category:doctor strange successfully
loading... category : hulk
loaded category:hulk successfully
loading... category : ironman
loaded category:ironman successfully
loading... category : loki
loaded category:loki successfully
loading... category : spider-man
loaded category:spider-man successfully
loading... category : thanos
loaded category:thanos successfully


In [3]:
test_image_paths = []
test_labels = []
# get all the image folder paths
Categories=['black widow', 'captain america', 'doctor strange', 'hulk', 'ironman', 'loki', 'spider-man', 'thanos']
datadir='dataset\\test' 

for i in Categories:
    
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        test_image_paths.append(os.path.join(path,img))
        test_labels.append(Categories.index(i))
    print(f'loaded category:{i} successfully')

loading... category : black widow
loaded category:black widow successfully
loading... category : captain america
loaded category:captain america successfully
loading... category : doctor strange
loaded category:doctor strange successfully
loading... category : hulk
loaded category:hulk successfully
loading... category : ironman
loaded category:ironman successfully
loading... category : loki
loaded category:loki successfully
loading... category : spider-man
loaded category:spider-man successfully
loading... category : thanos
loaded category:thanos successfully


In [35]:
scaler = StandardScaler()
def bag_of_words_SIFT(image_paths, codebook):
    orb = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        keypoints, descriptors = orb.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [36]:
codebook_SIFT = joblib.load('codebook_SIFT.joblib')
svm_SIFT = joblib.load('svm_bow_SIFT.joblib')

In [37]:
print('Generating BOW features for training set...')
train_images_SIFT = bag_of_words_SIFT(train_image_paths, codebook_SIFT)
train_images_scaled_SIFT = scaler.fit_transform(train_images_SIFT)
print('Train images:', train_images_SIFT.shape)

Generating BOW features for training set...
Train images: (2325, 150)


# Gaussian pixel noise

In [63]:
def bag_of_words_SIFT_GPN(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.gaussian_pixel_noise(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [68]:
l = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [69]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_GPN(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0
2
4
6
8
10
12
14
16
18


In [71]:
res

[0.27360969704337024,
 0.242403996988601,
 0.24732311116041233,
 0.23897523015191685,
 0.2515416060985023,
 0.23257588854623354,
 0.24964844879624748,
 0.2618438319684258,
 0.24748328684255932,
 0.24818010054385445]

In [72]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
0,0.273610
2,0.242404
4,0.247323
6,0.238975
8,0.251542
10,0.232576
12,0.249648
14,0.261844
16,0.247483
18,0.248180


In [73]:
df.to_csv("Gaussian pixel noise.csv")

# Gaussian blurring

In [75]:
def bag_of_words_SIFT_GB(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.gaussian_blur(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [76]:
res = []
for i in range(10):
    print(i)
    test_images = bag_of_words_SIFT_GB(test_image_paths, codebook_SIFT, i)
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0
1
2
3
4
5
6
7
8
9


In [77]:
d = {'f-1': res}
df = pd.DataFrame(data=d)
df

,f-1
0,0.273610
1,0.243160
2,0.232824
3,0.216480
4,0.209853
5,0.214553
6,0.195816
7,0.170992
8,0.189825
9,0.158790


In [79]:
df.to_csv("Gaussian blurring.csv")

# Image Contrast Increase 

In [80]:
def bag_of_words_SIFT_ICI(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.scale_contrast(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [81]:
l = [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.1, 1.15, 1.20, 1.25]

In [82]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_ICI(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

1.0
1.01
1.02
1.03
1.04
1.05
1.1
1.15
1.2
1.25


In [83]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
1.00,0.273610
1.01,0.273747
1.02,0.252256
1.03,0.257207
1.04,0.264634
1.05,0.250637
1.10,0.262991
1.15,0.265998
1.20,0.254205
1.25,0.252982


In [84]:
df.to_csv("Image Contrast Increase.csv")

# Image Contrast Decrease

In [87]:
l = [ 1.0, 0.95, 0.90, 0.85, 0.80, 0.60, 0.40, 0.30, 0.20, 0.10]

In [88]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_ICI(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

1.0
0.95
0.9
0.85
0.8
0.6
0.4
0.3
0.2
0.1


In [89]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
1.00,0.273610
0.95,0.257378
0.90,0.258881
0.85,0.265697
0.80,0.277105
0.60,0.257211
0.40,0.252932
0.30,0.246451
0.20,0.244595
0.10,0.253306


In [90]:
df.to_csv("Image Contrast Decrease.csv")

# Image Brightness Increase

In [91]:
def bag_of_words_SIFT_IB(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.change_brightness(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [92]:
l = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

In [93]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_IB(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0
5
10
15
20
25
30
35
40
45


In [94]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
0,0.273610
5,0.254829
10,0.257241
15,0.250382
20,0.223359
25,0.220670
30,0.217730
35,0.215704
40,0.205175
45,0.189511


In [95]:
df.to_csv("Image Brightness Increase.csv")

# Image Brightness Decrease

In [96]:
l = [0, -5, -10, -15, -20, -25, -30, -35, -40, -45]

In [97]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_IB(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0
-5
-10
-15
-20
-25
-30
-35
-40
-45


In [98]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
0,0.273610
-5,0.262279
-10,0.269541
-15,0.275715
-20,0.274209
-25,0.258727
-30,0.249542
-35,0.256700
-40,0.235373
-45,0.242433


In [99]:
df.to_csv("Image Brightness Decrease.csv")

# Occlusion of the Image Increase

In [100]:
def bag_of_words_SIFT_OII(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.occlusion(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [101]:
l = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

In [103]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_OII(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0
5
10
15
20
25
30
35
40
45


In [104]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
0,0.273610
5,0.258348
10,0.260781
15,0.241976
20,0.250839
25,0.263372
30,0.265080
35,0.249251
40,0.247592
45,0.258425


In [105]:
df.to_csv("Occlusion of the Image Increase.csv")

# Salt and Pepper Noise 

In [106]:
def bag_of_words_SIFT_SPN(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.salt_and_pepper(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [107]:
l =[0.00, 0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18]

In [108]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_SPN(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0.0
0.02
0.04
0.06
0.08
0.1
0.12
0.14
0.16
0.18


In [109]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
0.00,0.273610
0.02,0.263714
0.04,0.246110
0.06,0.205957
0.08,0.177894
0.10,0.171711
0.12,0.173848
0.14,0.166444
0.16,0.140485
0.18,0.133010


In [110]:
df.to_csv("Salt and Pepper Noise.csv")